In [18]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from ipynb.fs.full.preprocessing import parse_poems
from HMM_helper import obs_map_reverser

In [19]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()

# get words as integers and get the word map
all_words = []
words, word_map = parse_poems(text)
for i in range(len(words)):
    for j in range(len(words[i])):
        all_words.append(words[i][j])
words = all_words

# get reverse map
int_to_words = obs_map_reverser(word_map)
        
n_words = len(words)
n_vocab = len(word_map)

In [20]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 7
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_words - seq_length, 1):
    seq_in = words[i:i + seq_length]
    seq_out = words[i + seq_length]
    dataX.append([word for word in seq_in])
    dataY.append(seq_out)
    
n_sequences = len(dataX)

In [21]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [22]:
# define the LSTM model 
# (single layer with 200 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [24]:
# load the network weights from best checkpoint
filename = "weights-improvement-word-50-0.7077.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
def sample(preds, temp=1.0):
    '''
    Helper function for sampling from softmax with different temperatures.
    
    Inputs:
    preds: output of softmax function
    temp: temperature to scale by
    '''
    preds = np.asarray(preds).astype('float64')
    exp_preds = exp_preds = preds**(1/temp)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [32]:
# set seed for emissions
seed = "shall i compare thee to a summers"
seed = seed.split(' ')
seed_to_int = [word_map[word] for word in seed]

# generate emissions
pattern = seed_to_int
temps = [0.01, 0.1, 0.25, 0.75, 1.5]    
for temp in temps:    
    print('Temperature', temp)
    
    for j in range(14):
    
        # generate words
        for i in range(8):
            x = np.reshape(pattern, (1, len(pattern), 1))
            x = x / float(n_vocab)
            prediction = model.predict(x, verbose=0)[0]

            # sample according to temperature
            idx = sample(prediction, temp)

            result = int_to_words[idx]
            pattern.append(idx)
            pattern = pattern[1:len(pattern)]

            # output result 
            if i == 0:
                sys.stdout.write(result.capitalize() + ' ')
            else:
                sys.stdout.write(result + ' ')        
            if i == 7:
                sys.stdout.write('\n')
            sys.stdout.flush()

    print()

Temperature 0.01
Day loves fair eyes eye greater grief in 
And a but to in worth thou fresh 
Unthrifty to be and is thy youths thy 
The thou as sunken all thou fresh exchanged 
To the grown self mine of wanting my 
Not the tongue of when is my i 
Sweets seem denote suns them that not heart 
And one his engraft of who hath of 
Your am old plagues true that to whence 
Pen mine this imperfect one policy into to 
My love of my this am so thou 
Defects graces now till thee in that erred 
If and if overplus more than enough seem 
I that vex thee still to thy sweet 

Temperature 0.1
Will making addition and wilt hue but true 
Swiftfooted lay to should i and be tend 
Am yield meditation how his learneds wing by 
Write grace thee love voices not thy and 
Do all and thy tongue check by heart 
But swearing in my to outward of to 
Who graces i merits with heart name but 
When thy art of love and wont and 
Dregs to wit making his style admired every 
Vassal and this allow of churls at pleasure 
The